# Projeto deep learning

## DINO
Utilizando DINO para pré-treinar a rede

In [1]:
# Unpack dataset
import zipfile

path_to_zip_file = './datasets/chestx-ray14-v3.zip'
directory_to_extract_to = './datasets/chestx-ray14-v3'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [4]:
from torchvision import models as torchvision_models

torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))

torchvision_archs

['alexnet',
 'convnext_base',
 'convnext_large',
 'convnext_small',
 'convnext_tiny',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'googlenet',
 'inception_v3',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'regnet_x_16gf',
 'regnet_x_1_6gf',
 'regnet_x_32gf',
 'regnet_x_3_2gf',
 'regnet_x_400mf',
 'regnet_x_800mf',
 'regnet_x_8gf',
 'regnet_y_128gf',
 'regnet_y_16gf',
 'regnet_y_1_6gf',
 'regnet_y_32gf',
 'regnet_y_3_2gf',
 'regnet_y_400mf',
 'regnet_y_800mf',
 'regnet_y_8gf',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',

In [18]:
! pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 KB 8.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 KB 8.7 MB/s eta 0:00:0000:01
  Using cached tqdm-4.67.0-py3-none-any.whl (78 kB)


In [19]:
! python -m main_dino -h

Using cache found in /home/CIN/tta/.cache/torch/hub/facebookresearch_xcit_main
/home/CIN/tta/msc-deep-learning/deep_learning/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/CIN/tta/msc-deep-learning/deep_learning/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
usage: DINO [-h]
            [--arch {vit_tiny,vit_small,vit_base,xcit,deit_tiny,deit_small,alexnet,convnext_base,convnext_large,convnext_small,convnext_tiny,densenet121,densenet161,densenet169,densenet201,efficientnet_b0,efficientnet_b1,efficientnet_b2,efficientnet_b3,efficientnet_b4,efficientnet_b5,effic

In [24]:
! mkdir ./output/
! mkdir ./output/DINOXray
! ls

/bin/bash: line 1: mkdier: command not found
mkdir: cannot create directory ‘./output/DINOXray’: File exists
datasets		 eval_video_segmentation.py  README.md
deep_learning		 fine_tuning.ipynb	     results
dino.ipynb		 hubconf.py		     run_with_submitit.py
DINO_README.md		 __init__.py		     utils.py
eval_copy_detection.py	 LICENSE		     video_generation.py
eval_image_retrieval.py  main_dino.py		     vision_transformer.py
eval_knn.py		 output			     visualize_attention.py
eval_linear.py		 __pycache__


In [27]:
from torchvision.models import efficientnet_b0

# Load EfficientNet-B0 pre-trained on ImageNet
model = efficientnet_b0(pretrained=True)

# Access the feature extraction layers
feature_extractor = model.features

Sequential(
  (0): ConvNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): ConvNormActivation(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=F

In [41]:
model.classifier[1]

Linear(in_features=1280, out_features=1000, bias=True)

## Treinamento com o DINO-CXR

In [3]:
# Treinar EfficientNet pré-treinada com o  DINO
! python -m torch.distributed.launch --nproc_per_node=1 main_dino.py --arch efficientnet_b0 --pretrained True --optimizer adam --batch_size_per_gpu 64 --min_lr  0.00025 --weight_decay 1e-6 --weight_decay_end 1e-5 --num_workers 8 --out_dim 640 --data_path ./datasets/chestx-ray14-v3/train/ --output_dir ./output/DINOXray

/home/CIN/tta/msc-deep-learning/deep_learning/lib/python3.10/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Using cache found in /home/CIN/tta/.cache/torch/hub/facebookresearch_xcit_main
/home/CIN/tta/msc-deep-learning/deep_learning/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/CIN/tta/msc-deep-learning/deep_learning/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: F

## Salvar pesos do backbone

In [7]:
import torch
from torchvision.models import efficientnet_b0

def weights_update(model, checkpoint_path, key):
    checkpoint = torch.load(checkpoint_path)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in checkpoint[key].items() if k in model_dict}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    print(f'Checkpoint {checkpoint_path} was loaded')
    return model

save_model_path = './output/DINOXray'
model = efficientnet_b0()
model = weights_update(model, f"{save_model_path}/checkpoint.pth", 'teacher')

model_backbone_weights = model
print(model_backbone_weights)
torch.save({
            'model_state_dict': model_backbone_weights.state_dict(),
            }, f'{save_model_path}/efficientnet_b0_backbone_weights.ckpt')

Checkpoint ./output/DINOXray/checkpoint.pth was loaded
EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sig